# 1. Preprocesamiento de Datos (IL1 - Nivel c)
## Exploración inicial:
### Carga los datos y realiza una inspección inicial para entender su estructura, identificar valores faltantes y detectar outliers.
### Analiza la correlación entre las variables y los contaminantes.
## Transformaciones necesarias:
### Limpieza de datos: Manejo de valores faltantes (interpolación, imputación).
### Estandarización o normalización de variables.
### Codificación de variables categóricas, si las hay (One-Hot Encoding o similares).
### Creación de características adicionales basadas en relaciones conocidas (e.g., interacción entre variables meteorológicas y contaminantes).
### Selección de características relevantes:
### Utiliza técnicas como análisis de correlación, selección basada en importancia de características (e.g., usando Random Forest), o PCA si los datos son muy dimensionales.

# 2. Entrenamiento de Modelos de Regresión (IL2 - Nivel DESARROLLO)
## Selecciona al menos 5 modelos de regresión diferentes. Ejemplo:
- Regresión lineal.
- Árbol de decisión.
- Random Forest.
- XGBoost.
- Redes neuronales simples (MLP).

## Para cada modelo:
### Divide los datos en conjunto de entrenamiento (80%) y prueba (20%).
### Realiza una validación cruzada (k-fold) para evaluar el rendimiento en el conjunto de entrenamiento.
Optimiza los hiperparámetros mediante Grid Search o Random Search.
## Documenta:
### Por qué eliges cada modelo.
### Cómo ajustas los parámetros y los hiperparámetros.

3. Evaluación de la Capacidad Predictiva de los Modelos (IL3 - Nivel APLICACIÓN)
## Evalúa cada modelo utilizando métricas clave:
- MAE (Mean Absolute Error): Para medir la magnitud promedio de los errores.
- RMSE (Root Mean Squared Error): Para dar mayor peso a los errores más grandes.
- R² (Coeficiente de determinación): Para explicar la variabilidad de los datos.
## Genera gráficos comparativos:
- urvas de predicción frente a valores reales.
- Importancia de características para los modelos como Random Forest y XGBoost.
## Reflexiona sobre el rendimiento:
### ¿Qué modelos son más precisos y por qué?
### ¿Qué variables influyen más en los resultados?

# 4. Comparativa entre Modelos y Selección del Mejor Modelo (IL4 - Nivel APLICACIÓN)
### Presenta una tabla resumen con las métricas de cada modelo para cada contaminante.
## Fundamenta la selección del mejor modelo para cada contaminante:
### Considera precisión, velocidad de entrenamiento, y capacidad para generalizar.
## Discute las limitaciones de los modelos:
### ¿Hay suficiente representatividad en los datos?
### ¿Cómo podrían influir las características de los datos en el rendimiento?

# 5. Informe de Conclusiones y Reflexiones Finales (IL5 - Nivel DESARROLLO)
## Estructura sugerida:
## Introducción:
### Propósito del estudio.
### Descripción breve de los datos y contaminantes analizados.
## Preprocesamiento de datos:
### xplicación de las transformaciones realizadas y selección de características.
## Entrenamiento de modelos:
### Breve descripción de cada modelo y su optimización.
## Resultados y análisis:
### Evaluación detallada del rendimiento de cada modelo.
### Gráficos comparativos y tablas de métricas.
## Conclusiones:
### Selección del mejor modelo.
### eflexión sobre los resultados.
### Recomendaciones para mejorar futuros modelos.

# Importacciones, lectura y exploración de los datos. 

In [2]:
# import
import pandas as pd

# lectura del csv
path = 'data/dataset.csv'
data = pd.read_csv(path) 
#exploración de los datos 
data.info()

data.describe()

data.head(30)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10956 entries, 0 to 10955
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          10956 non-null  object 
 1   Ciudad        10956 non-null  object 
 2   NO (Ąg/m3)    10772 non-null  float64
 3   NO2 (Ąg/m3)   10723 non-null  float64
 4   NOX (Ąg/m3)   10731 non-null  float64
 5   PM10 (Ąg/m3)  8929 non-null   float64
 6   tavg          10950 non-null  float64
 7   tmin          10956 non-null  float64
 8   tmax          10956 non-null  float64
 9   prcp          10946 non-null  float64
 10  snow          283 non-null    float64
 11  wdir          8244 non-null   float64
 12  wspd          9522 non-null   float64
 13  wpgt          6301 non-null   float64
 14  pres          9139 non-null   float64
dtypes: float64(13), object(2)
memory usage: 1.3+ MB


,Date,Ciudad,NO (Ąg/m3),NO2 (Ąg/m3),NOX (Ąg/m3),PM10 (Ąg/m3),tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres
0,2015-01-01,BILBAO,31.0,51.0,99.0,NaN,6.0,-1.3,16.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,BILBAO,63.0,65.0,162.0,NaN,6.6,-0.9,15.8,0.0,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,BILBAO,46.0,56.0,126.0,28.0,7.7,0.3,15.2,0.0,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,BILBAO,18.0,43.0,70.0,21.0,11.3,7.3,13.7,0.0,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,BILBAO,51.0,54.0,132.0,26.0,8.7,3.4,18.0,0.0,NaN,NaN,NaN,NaN,NaN
5,2015-01-06,BILBAO,22.0,42.0,74.0,23.0,9.7,6.0,11.8,0.0,NaN,NaN,NaN,NaN,NaN
6,2015-01-07,BILBAO,37.0,50.0,106.0,24.0,9.0,3.1,16.9,0.0,NaN,NaN,NaN,NaN,NaN
7,2015-01-08,BILBAO,67.0,64.0,167.0,31.0,9.7,2.5,15.0,0.0,NaN,NaN,NaN,NaN,NaN
8,2015-01-09,BILBAO,65.0,60.0,159.0,33.0,10.8,4.4,20.1,0.0,NaN,NaN,NaN,NaN,NaN
9,2015-01-10,BILBAO,28.0,41.0,84.0,29.0,10.1,4.5,12.9,1.2,NaN,NaN,NaN,NaN,NaN
